In [7]:
import face_recognition
from PIL import Image
import cv2
import numpy as np
import paramiko

# This is a super simple (but slow) example of running face recognition on live video from your webcam.
# There's a second example that's a little more complicated but runs faster.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

#先自动开启ssh，启动摄像头
# SSH连接信息
hostname = '192.168.137.32'  # 树莓派的IP地址
port = 22  # SSH端口号，默认为22
username = 'wwb2'  # 树莓派的用户名
password = 'pi'  # 树莓派用户的密码

# 创建SSH客户端对象
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # 连接到树莓派
    client.connect(hostname, port, username, password)
    
    # 执行sudo motion命令
    stdin, stdout, stderr = client.exec_command('sudo motion')

    # 获取命令执行结果
    output = stdout.read().decode('utf-8')
    print(output)
    
    # 关闭SSH连接
    client.close()

except paramiko.AuthenticationException:
    print("认证失败，请检查用户名和密码。")

except paramiko.SSHException as ssh_err:
    print("SSH连接错误: ", ssh_err)

except paramiko.Exception as e:
    print("错误: ", e)


# 定义视频源的URL
url = "http://192.168.137.32:8081/"#树莓派1，用于面部识别

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(url)


# Load a sample picture and learn how to recognize it.
# lwz_image = face_recognition.load_image_file("lwz.jpg")
lwz_image = Image.open("lwz.jpg")
lwz_image = lwz_image.convert("RGB")
lwz_face_encoding = face_recognition.face_encodings(np.array(lwz_image))[0]

# Load a second sample picture and learn how to recognize it.
# wwb_image = face_recognition.load_image_file("wwb.jpg")
wwb_image = Image.open("wwb.jpg")
wwb_image = wwb_image.convert("RGB")
wwb_face_encoding = face_recognition.face_encodings(np.array(wwb_image))[0]

# Load a second sample picture and learn how to recognize it.
# wwb_image = face_recognition.load_image_file("wwb.jpg")
zjy_image = Image.open("zjy.jpg")
zjy_image = zjy_image.convert("RGB")
zjy_face_encoding = face_recognition.face_encodings(np.array(zjy_image))[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    lwz_face_encoding,
    wwb_face_encoding,
    zjy_face_encoding
]
known_face_names = [
    "LIN WUZHEN",
    "WANG WEIBO",
    "ZHANG JINGYUAN"
    
]

# Set the counter value
frame_count = 0
# Set the interval to process face recognition
interval = 5

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 0)
    # 改变摄像头图像的大小，图像小，所做的计算就少
    frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Increment the frame counter
    frame_count += 1
    
    if frame_count % interval == 0:
        
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face enqcodings in the frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (225, 0, 0), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 17), (right, bottom), (255 , 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()